<a href="https://www.kaggle.com/code/derekxue/humanproteincell?scriptVersionId=141032367" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Read through the following notebooks and links. Thanks all for their briliant ideas.

https://www.kaggle.com/c/hpa-single-cell-image-classification/discussion/221550 

https://www.kaggle.com/code/dragonzhang/fastai-cell-tile-prototyping-training/notebook  

https://www.kaggle.com/code/p4rallax/using-albumentations-with-fast-ai-datablock-api  

## Use fastai framework
## Load datasets, build dataloaders
## Use load_from_df
## Integrate albumentation. Use seamless augmentation. 

In [1]:
!pip install fastai -Uqq 

In [2]:
import pandas as pd
import numpy as np
import PIL.Image as Image
from fastai.data.all import *
from fastai.vision.all import *
from fastai.tabular.all import *

import albumentations as A

import warnings
# Ignore warning messages
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
protein_source = Path('/kaggle/input/hpa-single-cell-image-classification')

In [4]:
# build an image file meta data frame. 
%timeit
meta_df = pd.DataFrame({
    'image':(
        *get_image_files(protein_source),
    )
})

meta_df.head()

,image
0,/kaggle/input/hpa-single-cell-image-classification/test/9b1d4b27-6946-4b86-a818-8e91029c3dfa_red.png
1,/kaggle/input/hpa-single-cell-image-classification/test/5af9bb81-719a-4374-9a12-5e5665329df6_blue.png
2,/kaggle/input/hpa-single-cell-image-classification/test/8adb17b7-ce2c-4721-bf34-c1806c72b4d4_yellow.png
3,/kaggle/input/hpa-single-cell-image-classification/test/c6379d83-6b05-4a29-8f56-05a381d94d2a_blue.png
4,/kaggle/input/hpa-single-cell-image-classification/test/277b3f6d-099b-4b6d-8592-a06ad6f52beb_red.png


In [5]:
# Get the parent folder name of each image path
image_paths = meta_df["image"].tolist()
parent_folder_names = []
for image_path in image_paths:
    parent_folder_name = os.path.dirname(image_path)
    label_value = parent_folder_name.split("/")[-1]
    parent_folder_names.append(label_value)

# Add the "trn_tst" column to the DataFrame
meta_df["trn_tst"] = parent_folder_names

# check for labels
meta_df['trn_tst'].value_counts()

train    87224
test      2236
Name: label, dtype: int64

### Get image id

In [6]:
def get_image_id(image_file): return str(image_file).split('_')[0]

In [7]:

meta_df['image_id'] = meta_df['image'].apply(get_image_id)

meta_df.head()

,image,label,image_id
0,/kaggle/input/hpa-single-cell-image-classification/test/9b1d4b27-6946-4b86-a818-8e91029c3dfa_red.png,test,/kaggle/input/hpa-single-cell-image-classification/test/9b1d4b27-6946-4b86-a818-8e91029c3dfa
1,/kaggle/input/hpa-single-cell-image-classification/test/5af9bb81-719a-4374-9a12-5e5665329df6_blue.png,test,/kaggle/input/hpa-single-cell-image-classification/test/5af9bb81-719a-4374-9a12-5e5665329df6
2,/kaggle/input/hpa-single-cell-image-classification/test/8adb17b7-ce2c-4721-bf34-c1806c72b4d4_yellow.png,test,/kaggle/input/hpa-single-cell-image-classification/test/8adb17b7-ce2c-4721-bf34-c1806c72b4d4
3,/kaggle/input/hpa-single-cell-image-classification/test/c6379d83-6b05-4a29-8f56-05a381d94d2a_blue.png,test,/kaggle/input/hpa-single-cell-image-classification/test/c6379d83-6b05-4a29-8f56-05a381d94d2a
4,/kaggle/input/hpa-single-cell-image-classification/test/277b3f6d-099b-4b6d-8592-a06ad6f52beb_red.png,test,/kaggle/input/hpa-single-cell-image-classification/test/277b3f6d-099b-4b6d-8592-a06ad6f52beb


In [8]:
# Check for unique image id in train dataset
meta_df[meta_df['trn_tst'] == 'train']['image_id'].nunique()

21806

In [9]:
# Check for unique image id in train dataset
meta_df[meta_df['trn_tst'] == 'test']['image_id'].nunique()

559

In [10]:
len(meta_df) / 4  #all images have R G B Yellow .png files

22365.0

In [16]:
#Reduce to a small set.
%time
meta_df = meta_df[meta_df['trn_tst'] == 'train']
meta_small = meta_df.iloc[:400, :].copy()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 16.5 µs


In [17]:
def get_shape(image_file):
    image = Image.open(image_file)
    return image.shape

In [18]:
%time
meta_small['shape'] = meta_small['image'].apply(get_shape)
meta_small.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


,image,label,image_id,shape
6306,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_blue.png,train,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"(2048, 2048)"
37394,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_green.png,train,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"(2048, 2048)"
78257,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_red.png,train,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"(2048, 2048)"
79984,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_yellow.png,train,/kaggle/input/hpa-single-cell-image-classification/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"(2048, 2048)"
17045,/kaggle/input/hpa-single-cell-image-classification/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0_blue.png,train,/kaggle/input/hpa-single-cell-image-classification/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,"(2048, 2048)"


### Stack R, B, G, Yellow  


https://www.kaggle.com/datasets/philculliton/hpa-challenge-2021-extra-train-images/code  

In [19]:
import matplotlib.pyplot as plt
import cv2

from skimage.io import imsave, imread
import tqdm

In [21]:
meta_small.iloc[0, 2].split('/')[-1]  #demo

'000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0'

In [22]:
%time
SZ = 2048
i=0
for iid in tqdm.tqdm(meta_small['image_id']):
    r = cv2.imread(f'{iid}_red.png', 0)
    g = cv2.imread(f'{iid}_green.png', 0)
    b = cv2.imread(f'{iid}_blue.png', 0)
    a = cv2.imread(f'{iid}_yellow.png', 0)
    if not r.shape[0] == SZ:
        r = cv2.resize(r, (SZ, SZ))
        g = cv2.resize(g, (SZ, SZ))
        b = cv2.resize(b, (SZ, SZ))
        a = cv2.resize(a, (SZ, SZ))
    img = np.stack([r, g, b, a], -1)
    fname = meta_small.iloc[i, 2].split('/')[-1]
    imsave(f'./{fname}_.png', img)
    i += 1

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.7 µs


100%|██████████| 400/400 [16:16<00:00,  2.44s/it]


In [23]:
df_train = pd.read_csv('/kaggle/input/hpa-single-cell-image-classification/train.csv')
df_train.head()

,ID,Label
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0


In [24]:
# build an image file meta data frame. 
meta_sml_wk = pd.DataFrame({
    'image':(
        *get_image_files('/kaggle/working/'),
    )
})

meta_sml_wk.head()

,image
0,/kaggle/working/00a71d92-bbb2-11e8-b2ba-ac1f6b6435d0_.png
1,/kaggle/working/001838f8-bbca-11e8-b2bc-ac1f6b6435d0_.png
2,/kaggle/working/009b8c20-bbc6-11e8-b2bc-ac1f6b6435d0_.png
3,/kaggle/working/004d8a0e-bbc4-11e8-b2bc-ac1f6b6435d0_.png
4,/kaggle/working/012c02f2-bbab-11e8-b2ba-ac1f6b6435d0_.png


In [25]:
def get_image_id_wk(image_file): 
    tmp = str(image_file).split('_')[0]
    return tmp.split('/')[-1]

In [26]:
meta_sml_wk['ID'] = meta_sml_wk['image'].apply(get_image_id_wk)

In [46]:
df_merge = pd.merge(meta_sml_wk, df_train, left_on='ID', right_on='ID', how='outer')

In [47]:
df_merge = df_merge.dropna()

### Process label. Convert integer back to a string

https://www.kaggle.com/code/lnhtrang/hpa-public-data-download-and-hpacellseg/notebook  

In [55]:
# All label names in the public HPA and their corresponding index. 
all_labels = {
    "Nucleoplasm": 0,
    "Nuclear membrane": 1,
    "Nucleoli": 2,
    "Nucleoli fibrillar center": 3,
    "Nuclear speckles": 4,
    "Nuclear bodies": 5,
    "Endoplasmic reticulum": 6,
    "Golgi apparatus": 7,
    "Intermediate filaments": 8,
#     "Actin filaments": 9,
    "Focal adhesion sites": 9,
    "Microtubules": 10,
    "Mitotic spindle": 11,
    "Centrosome": 12,
#     "Centriolar satellite": 12,
    "Plasma membrane": 13,
#     "Cell Junctions": 13,
    "Mitochondria": 14,
    "Aggresome": 15,
    "Cytosol": 16,
    "Vesicles": 17,
#     "Peroxisomes": 17,
#     "Endosomes": 17,
#     "Lysosomes": 17,
#     "Lipid droplets": 17,
#     "Cytoplasmic bodies": 17,
    "No staining": 18
}

In [50]:
df_merge['Label']

0               [13]
1               [12]
2                [7]
3            [16, 2]
4                [8]
           ...      
95            [1, 0]
96               [5]
97    [16, 1, 2, 13]
98           [16, 8]
99           [14, 0]
Name: Label, Length: 100, dtype: object

In [49]:
def g(df_merge):
    for i in range(len(df_merge)):
        label = df_merge.loc[i, 'Label']
        if '|' in label:
            df_merge.loc[i, 'Label'] = label.split('|')
        else:
            df_merge.loc[i, 'Label'] = [label]
    return df_merge

df_merge = g(df_merge.copy())
print(df_merge['Label'].head())

0       [13]
1       [12]
2        [7]
3    [16, 2]
4        [8]
Name: Label, dtype: object


In [56]:
value = {i for i in all_labels if all_labels[i]==9}
print("key by value:",value)

key by value: {'Focal adhesion sites'}


### Use albumentation

In [ ]:
class AlbumentationsTransform(DisplayedTransform):
    split_idx,order=0,2
    def __init__(self, train_aug): store_attr()
    
    def encodes(self, img: PILImage):
        aug_img = self.train_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

### Integrate albumentation. Use seamless augmentation

In [ ]:
def get_train_aug(): return albumentations.Compose([
            #albumentations.HueSaturationValue(
                #hue_shift_limit=0.2, 
                #sat_shift_limit=0.2, 
                #val_shift_limit=0.2, 
                #p=0.5
            #),
            A.CoarseDropout(p=0.5),
            A.RandomContrast(p = 0.6),
            A.ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
            A.RandomRotate90(p=1),
    
])

In [ ]:
item_tfms = [RandomResizedCrop(224, min_scale=0.75, ratio=(1.,1.)),
             AlbumentationsTransform(get_train_aug())]
batch_tfms = [*aug_transforms(flip_vert=True, size=128, max_warp=0),  
              Normalize.from_stats(*sample_stats)]
bs=16

In [ ]:
labels = [str(i) for i in range(19)]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(vocab=labels)),
                splitter=ColSplitter(col='is_valid'),
                get_x=get_x,
                get_y=get_y,
                item_tfms=item_tfms,
                batch_tfms=batch_tfms
                )
dls = dblock.dataloaders(dfs, bs=bs)

In [ ]:
dls.show_batch(nrows=3, ncols=3)